In [1]:
# This is the template for the submission. If you want, you can develop your algorithm in a regular Python script and copy the code here for submission.

# Team members (e-mail, legi):
# chozhang@ethz.ch, 22-945-562
# minghli@student.ethz.ch, 22-953-293
# changli@student.ethz.ch, 22-944-474

In [2]:
# If you are in Kaggle, first go to notebook settings and add data from the stepcount competition
# The competition name is: Mobile Health 2023 - Step Count Submission
# After importing the dataset, run this code block to add Lilygo to the path
import os
import sys
curr_environ = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', 'Localhost')
if curr_environ != 'Localhost': 
    sys.path.append("/kaggle/input")
    input_dir = '/kaggle/input/mobile-health-2023-step-count'
else:
    input_dir = os.path.abspath('')

In [3]:
import pandas as pd
import numpy as np
from scipy import signal

from Lilygo.Recording import Recording
from Lilygo.Dataset import Dataset
from os import listdir
from os.path import isfile, join

# import matplotlib.pyplot as plt
# %matplotlib inline

In [4]:
# for tuning parameters
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator

In [34]:
# Get the path of all traces
dir_traces = f'{input_dir}/data/traces'
dir_labeled_traces = f'{input_dir}/data/labeled_traces'
traces_fnames = [join(dir_traces, f) 
          for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
labeled_traces_fnames = [join(dir_labeled_traces, f) 
                  for f in listdir(dir_labeled_traces) if isfile(join(dir_labeled_traces, f)) and not 'csv' in f]
traces_fnames.sort()
labeled_traces_fnames.sort()

In [22]:
df = pd.read_csv(f'{input_dir}/data/labeled_traces/labels.csv')
expected_counts = df['Expected'].to_list()

In [6]:
### signal processing functions ###
def parse(signal, ds_freq:float=20.0, zero_mean:bool=False):
    """downsampling the signal to specific frequency ds_freq, and make the data with zero mean if zero_mean is True"""
    ori_time_seq = np.array(signal.timestamps)
    ori_value_seq = np.array(signal.values)
    if zero_mean: ori_value_seq = ori_value_seq - np.mean(ori_value_seq)
    dt = 1./ds_freq
    time_seq = np.arange(start=np.min(ori_time_seq), stop=np.max(ori_time_seq), step=dt)
    value_seq = np.interp(time_seq, ori_time_seq, ori_value_seq)
    return time_seq, value_seq
    
def bp_filter(value_seq, fp:float=3, fs:float=20.0):
    """apply band pass filter to the sequence. fp is the threshold frequency, and fs is the sampling frequency."""
    sos = signal.butter(N=4, Wn=[0.5,fp], btype='bandpass', fs=fs, output='sos')
    filtered = signal.sosfilt(sos, value_seq)
    return filtered
    
def get_envelop(value_seq, 
                fs:float=20, 
                half_window_size:float=0.5, 
                _min:float=20., 
                _max:float=500.):
    """
    get the envelop as the adaptive local norm of the signal, currently the mode of vector (no negative values).
    the envelop is calculated by the maximum in a window, half_window_size is the seconds of time.
    _min and _max for clip. designed for gyro. not tuned for acceleration.
    """
    half_win = int(fs*half_window_size)
    seq = np.concatenate([np.zeros((half_win,)),value_seq,np.zeros((half_win,))])
    envelop = np.array([np.max(seq[k-half_win:k+half_win+1]) for k in range(half_win,half_win+len(value_seq))])
    return np.clip(envelop, _min, _max)

In [36]:
traces = [Recording(f, no_labels=True, mute=True) for f in traces_fnames]
# not sure if I should change no_labels to False
labeled_traces = [Recording(f, no_labels=True, mute=True) for f in labeled_traces_fnames]

In [64]:
class StepCounter(BaseEstimator):
    def __init__(self, acc_min=1.,  acc_max=3.,    acc_height=0.25,
                       gyro_min=20., gyro_max=500., gyro_height=0.5,
                       acc_weight=0.9):
        self.acc_min = acc_min
        self.acc_max = acc_max
        self.acc_height = acc_height
        self.gyro_min = gyro_min
        self.gyro_max = gyro_max
        self.gyro_height = gyro_height
        self.acc_weight = acc_weight

    def fit(self, data, labels):
        # no learning actually, just to fit the estimator interface
        return self
    
    def score(self, X, y_true, sample_weight=None, normalize=True) -> float:
        '''
        Get the "score" of the step counting result. 
        The score is calculated based on how different the step count is from the true values
        '''
        y_predicted = self.predict(X)
        diff = y_predicted - y_true
        scores = np.zeros(len(diff))
        for i in range(len(diff)):
            s = 1 - abs(diff[i]) / y_true[i]
            if s < 0:
                s = 0
            scores[i] = s
        if normalize:
            return np.average(scores, weights=sample_weight)
        elif sample_weight is not None:
            return np.dot(scores, sample_weight)
        else:
            return scores.sum()
        
    def predict(self, traces):
        res = np.zeros(len(traces))
        i = 0
        for trace in traces:
            data = trace.data
            # accelerator data
            ax, ay, az = data['ax'], data['ay'], data['az']
            # gyroscope data
            gx, gy, gz = data['gx'], data['gy'], data['gz']
            # magnetometer data
            mx, my, mz = data['mx'], data['my'], data['mz']
            # temperature, in celsius degrees
            temp = data['temperature']
            acc_step_counts = self._count_steps(ax, ay, az,
                                               _max=self.acc_max,
                                               _min=self.acc_min,
                                               _height=self.acc_height)
            gyro_step_counts = self._count_steps(gx, gy, gz,
                                                _max=self.gyro_max,
                                                _min=self.gyro_min,
                                                _height=self.gyro_height)
            res[i] = int(self.acc_weight * acc_step_counts +
                        (1.0 - self.acc_weight) * gyro_step_counts)
            i += 1
        if len(res) == 1:
            return res[0]
        else:
            return res
    
    def _count_steps(self, ax, ay, az, _max, _min, _height):
        # interval of m and temp: 80ms; others 50ms
        # acc are in unit "g". gyro should be within -255, 255
        g_t, gx_v = parse(ax)  # use acceleration seems better.
        _, gy_v = parse(ay)
        _, gz_v = parse(az)
        
        # calculate the mode.
        g_v = np.sqrt(np.sum(np.square([gx_v,gy_v,gz_v]),axis=0))  
        g_v /= get_envelop(g_v, 
                           _min=_min, 
                           _max=_max)  # an adaptive local norm
        # band pass
        filtered_gv = bp_filter(g_v)
        # amp 1/4 after filtering, should be amplified 4x.
        filtered_gv = filtered_gv * (filtered_gv > 0) * 4

        # 0.5 optimal for gyro. not tuned for acc but I am lazy. 
        peaks, _ = signal.find_peaks(filtered_gv, 
                                     height=_height, 
                                     distance=20 * 0.2)
        # when _min=20 for acc, height=0.01 looks good. sota: _min=1, height=0.25
        step_count = len(peaks)  # peaks are the steps.
        return step_count

In [ ]:
# This code block is for parameter tuning and will take decades to finish
# do not run unless you want to tune param
step_counter = StepCounter()
parameters = {
    "acc_height":  np.arange(0.05, 1.0, step=0.05),
    "gyro_height": np.arange(0.05, 1.0, step=0.05),
    "acc_weight":  np.arange(0.1, 1.0, step=0.1)
}
param_tuner = GridSearchCV(step_counter, parameters, cv=len(labeled_traces))
param_tuner.fit(labeled_traces, expected_counts)

print(param_tuner.best_score_)
print(param_tuner.best_params_)

Below is the template

In [ ]:
# Get the path of all traces
dir_traces = '/kaggle/input/mobile-health-2023-step-count/data/traces'
filenames = [join(dir_traces, f) for f in listdir(dir_traces) if isfile(join(dir_traces, f))]
filenames.sort()

In [19]:
# Loop through all traces and calculate the step count for each trace
stepCounts = []
ids = []
best_step_counter = StepCounter(param_tuner.best_params_)
for filename in filenames:
    trace = Recording(filename, no_labels=True, mute=True)
    stepCount = 0  # This is your variable of the step counting that you need to change

    #
    # Your algorithm goes here
    # Make sure, you only use data from the LilyGo Wristband, namely the following 10 keys (as in trace.data[key]):
    # 3-axis accelerometer: key in [ax, ay, az]
    # 3-axis gyro: key in [gx, gy, gz]
    # 3-axis magnetometer: key in [mx, my, mz]
    # IMU temperature: key==temperature
    #
    stepCount = best_step_counter.predict(trace)

    # Append your calculated step counts and the id of the trace to the corresponding array
    stepCounts.append(stepCount)
    ids.append(filename.split('_')[1][:2])
print(stepCounts)

0.5856884404489839 1.5119752078742092
101
0.6717981163008813 1.636599812667238
202
0.5771699336371727 1.6637434449318385
103
0.2672999547148938 2.3801752797203
106
0.5357651728415087 2.0651952544566683
108
0.291490096937427 3.462568824888634
117
0.6637231815515947 3.377970236027716
123
0.45600722893016454 3.463995960038493
36
0.4217628406965969 3.276594955079884
155
0.3367159277149306 1.9826480560934336
69


In [20]:
# Write the detected step counts into a .csv file to then upload the .csv file to Kaggle
# When cross-checking the .csv file on your computer, we recommend using the text editor and NOT excel so that the results are displayed correctly
# IMPORTANT: Do NOT change the name of the columns ('Id' and 'Predicted') of the .csv file
df = pd.DataFrame({'Id':ids,'Predicted':stepCounts})
df.to_csv('/kaggle/working/submission.csv', index=False)